In [1]:
#encoding=utf-8
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

# get tfidf

In [2]:
# path = '/home/kai/data/jiangning/Chatbot_1/Chatbot1.0/data/'
path = '../../data/IDClassifier/'
data = pd.read_csv(path + 'cleaned_mock_up_data.csv', encoding='utf8')
print(data.shape)
data.head(10)

(4575, 2)


,label,split_text
0,2,啊 ？ 你 说 啥子
1,0,是 哈萨克斯坦
2,1,你 有 错 的 人
3,2,不要 打扰 打 这个 电话 。
4,0,我 在 做 什么 ？
5,2,你 的 大脑 生病 了
6,0,等 一下 回到 你 身边
7,1,这 是 你 的 祖父
8,2,没有 听 清楚
9,2,天气 好


In [3]:
# phrase_vectorizer1 = TfidfVectorizer(ngram_range=(1,3),
#                                     strip_accents='unicode', 
#                                     max_features=100000, 
#                                     analyzer='word',
#                                     sublinear_tf=True,
#                                     token_pattern=r'\w{1,}')
# data.split_text.iloc[:2]
# dd = ['I like apple do you like it', 'I do not like banana']
# phrase_vectorizer1.fit(dd)

# print('transform phrase')
# phrase = phrase_vectorizer1.transform(dd)


In [4]:
data.split_text.iloc[1]

'是   哈萨克斯坦'

In [5]:
# phrase_vectorizer1.vocabulary_

In [6]:
phrase_vectorizer = TfidfVectorizer(ngram_range=(1,3),
                                    strip_accents='unicode', 
                                    max_features=100000, 
                                    analyzer='word',
                                    sublinear_tf=True,
                                    token_pattern=r'\w{1,}')

print('fitting phrase')
phrase_vectorizer.fit(data.split_text)

print('transform phrase')
phrase = phrase_vectorizer.transform(data.split_text)

phrase

fitting phrase
transform phrase


<4575x7659 sparse matrix of type '<class 'numpy.float64'>'
	with 36875 stored elements in Compressed Sparse Row format>

# SVC

In [7]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC

l_svc = LinearSVC()
clf = CalibratedClassifierCV(l_svc) 
clf.fit(phrase, data.label)
print(clf.score(phrase, data.label))

0.939672131148


In [8]:
print(clf.predict(phrase))

[2 0 1 ..., 0 1 0]


# logistic

In [9]:
from sklearn.linear_model import LogisticRegression

In [10]:
log_r = LogisticRegression()
log_r.fit(phrase, data.label)
print(log_r.score(phrase, data.label))

0.91737704918


In [11]:
print(log_r.predict(phrase))

[2 0 1 ..., 0 1 0]


In [12]:
# l_svc.predict(phrase)-log_r.predict(phrase)

# lightGBM

In [13]:
import lightgbm as lgb

In [14]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(data.label.values)
onelabels = le.transform(data.label.values)

In [15]:
onelabels

array([2, 0, 1, ..., 0, 1, 0])

In [16]:
from sklearn.preprocessing import MultiLabelBinarizer
multicoder = MultiLabelBinarizer()
lables = multicoder.fit_transform([data.label.values])

In [17]:
lables

array([[1, 1, 1]])

In [18]:
data.label.values

array([2, 0, 1, ..., 0, 1, 0])

In [19]:
# lables.todense()

In [20]:
params = {
    'learning_rate': 0.2,
    'num_iterations':1000,
    'application': 'multiclassova',
    'num_class': 3,
    'num_leaves': 31,
    'verbosity': -1,
    'metric': 'multi_error',
    'data_random_seed': 2,
#     'bagging_fraction': 0.8,
#     'feature_fraction': 0.6,
    'nthread': 4,
    'lambda_l1': 1,
    'lambda_l2': 1,
    'early_stopping_round':200
} 

# lgbm_train = lgb.Dataset(phrase, data.label)
lgbm_train = lgb.Dataset(phrase, onelabels)
lgbm_val = lgb.Dataset(phrase, onelabels)
lgbm_model = lgb.train(params,lgbm_train,valid_sets=lgbm_val, verbose_eval=5)

/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:104: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 200 rounds.
[5]	valid_0's multi_error: 0.215519
[10]	valid_0's multi_error: 0.203497
[15]	valid_0's multi_error: 0.195191
[20]	valid_0's multi_error: 0.183825
[25]	valid_0's multi_error: 0.174426
[30]	valid_0's multi_error: 0.170055
[35]	valid_0's multi_error: 0.162842
[40]	valid_0's multi_error: 0.158689
[45]	valid_0's multi_error: 0.153005
[50]	valid_0's multi_error: 0.151257
[55]	valid_0's multi_error: 0.147322
[60]	valid_0's multi_error: 0.146667
[65]	valid_0's multi_error: 0.144699
[70]	valid_0's multi_error: 0.142951
[75]	valid_0's multi_error: 0.141421
[80]	valid_0's multi_error: 0.140984
[85]	valid_0's multi_error: 0.139016
[90]	valid_0's multi_error: 0.138798
[95]	valid_0's multi_error: 0.137049
[100]	valid_0's multi_error: 0.135956
[105]	valid_0's multi_error: 0.135738
[110]	valid_0's multi_error: 0.135519
[115]	valid_0's multi_error: 0.135738
[120]	valid_0's multi_error: 0.135301
[125]	valid_0's multi_error: 0.135082
[130]	v

In [21]:
print(lgbm_model.predict(phrase))

[[ 0.02887649  0.0016278   0.96811748]
 [ 0.96606695  0.01852371  0.0051752 ]
 [ 0.00201729  0.98810065  0.02103302]
 ..., 
 [ 0.99878825  0.00301458  0.00181814]
 [ 0.00277956  0.99492175  0.00271279]
 [ 0.98699805  0.02386025  0.02089204]]


In [22]:
# lgbm_model.predict(phrase)

In [23]:
phrase

<4575x7659 sparse matrix of type '<class 'numpy.float64'>'
	with 36875 stored elements in Compressed Sparse Row format>

# Save

In [24]:
import pickle

# save tfidf
pickle.dump(phrase_vectorizer, open("../../savedModel/IDClassifier/tfidf.pickle", "wb"))
# pickle.dump(train_comment_features, open("train_comment_features.pickle", "wb"))
# pickle.dump(test_comment_features, open("test_comment_features.pickle", "wb"))

# save linear svc
pickle.dump(clf, open("../../savedModel/IDClassifier/LinearSVC.pickle", "wb"))
# save logistic
pickle.dump(log_r, open("../../savedModel/IDClassifier/Logistic.pickle", "wb"))
# save lightGBM
pickle.dump(lgbm_model, open("../../savedModel/IDClassifier/Lgbm.pickle", "wb"))

In [25]:
# pickle.load(open("../../savedModel/IDClassifier/Lgbm.pickle", 'rb'))

# test

In [37]:
test = phrase_vectorizer.transform(['是 呀'])
# test = phrase_vectorizer.transform(['我 在 洗澡'])


In [38]:
test

<1x7659 sparse matrix of type '<class 'numpy.float64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [39]:
clf.predict_proba(test) # linear svc

array([[ 0.86230754,  0.06565965,  0.0720328 ]])

In [40]:
log_r.predict_proba(test) # logistic

array([[ 0.65950004,  0.15774823,  0.18275174]])

In [41]:
lgbm_model.predict(test) # light gbm

array([[ 0.96606695,  0.01852371,  0.0051752 ]])

# labeling

In [42]:
# basic logic: find the max probability of 3 models, if it is larger than threshold, return the corresponding label, otherwise, return 2 (others).
result = np.vstack((clf.predict_proba(test),log_r.predict_proba(test),lgbm_model.predict(test)))
pos = np.where(result == np.max(result))

threshold = 0.7
if np.max(result)<threshold:
    label = 2
else:
    label = pos[1]
    label = label[0]
    
print('label=',label)
print('prob=',np.max(result))

label= 0
prob= 0.966066947334


# Save the whole thing

In [43]:
# import jieba
# import numpy as np

# class IDClassifier:
    
#     def __init__(self, **model):
#         """
#         suggested parameters:
#         svc, logistic, lightgbm, jieba_path,tfidf
#         """
#         self._load_model(**model)
        
#     def _load_model(self,**model):
#         self.svc = model.get('svc')
#         self.logistic = model.get('logistic')
#         self.lightgbm = model.get('lightgbm')
#         self.tfidf = model.get('tfidf')
#         # load jieba
#         jieba_path = model.get('jieba_path')
#         if jieba_path is not None:
#             jieba.load_userdict(jieba_path)
        
        
#     def classify(self, sentence):
#         sentence = jieba.cut(sentence, cut_all = False)
#         sentence = ' '.join(sentence)
#         matrix = self.tfidf.transform([sentence])
#         result = np.vstack((self.svc.predict_proba(matrix),
#                             self.logistic.predict_proba(matrix),
#                             self.lightgbm.predict(matrix)))
#         max_pred = np.max(result, axis=0)
#         max_arg = np.argmax(max_pred)
#         threshold = 0.6
#         if np.max(max_pred<threshold):
#             label = 2
#         else:
#             label = max_arg
#         return (label, np.max(max_pred))
     
        

In [45]:
%load_ext autoreload
%autoreload 2

In [46]:
from IDClassifier_py import IDClassifier
idc = IDClassifier(svc=clf, logistic=log_r, lightgbm=lgbm_model, tfidf=phrase_vectorizer, jieba_path='../WordCut/userdict.txt')

pickle.dump(idc, open("../../savedModel/IDClassifier/IDClassifier.pickle", "wb"))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.455 seconds.
Prefix dict has been built succesfully.


In [47]:
## when apply, need to import sys path of the .py file, then load pkl file 